In [87]:
import glob
import pandas as pd
import re

In [88]:
path = 'tmp/FATCAT_aligments_twisted/'
files = glob.glob(path + '*.txt')

In [89]:
#extracting relevant information from the files

lines_with_tmscore = []

for file in files:
    file_and_data = [file]

    with open(file, 'r') as f:
        for line in f:
            if line.startswith('TM-score=') | line.startswith('Aligned length=') | line.startswith('Length of Chain'):
                
                file_and_data.append( line.strip() )
        
    lines_with_tmscore.append(file_and_data)



In [90]:
#creating dataframe with the information
df = pd.DataFrame(lines_with_tmscore, columns=['File', 'Length_Chain1', 'Length_Chain2', 'Aligned length', 'TM-score_Chain1', 'TM-score_Chain2'])

In [91]:
#extracting columns and converting to numeric
df[['pdb1', 'pdb2']] = (df
 .File
 .str.split('/', expand=True).iloc[:, -1]
 .str.split('_', expand=True).iloc[:,:2]
 )

In [92]:
df[['Aligned_length', 'RMSD', 'Seq_ID']] = df['Aligned length'].str.split(',', expand=True)

In [93]:
#regluar expression for each column to avoid problems
#extracting length of chains
df['Length_Chain1'] = df['Length_Chain1'].str.extract(r'(\s+\d+\.?\d*)')
df['Length_Chain2'] = df['Length_Chain2'].str.extract(r'(\s+\d+\.?\d*)')

#extracting aligned length, RMSD, Seq_ID
df['Aligned_length'] = df['Aligned_length'].str.extract(r'(\s+\d+\.?\d*)')
df['RMSD'] = df['RMSD'].str.extract(r'(\s+\d+\.?\d*)')
df['Seq_ID'] = df['Seq_ID'].str.extract(r'(\s+\d+\.?\d*)')

#extracting TMscores
df['TM-score_Chain1'] = df['TM-score_Chain1'].str.extract(r'(\s+\d+\.?\d*\s+)')
df['TM-score_Chain2'] = df['TM-score_Chain2'].str.extract(r'(\s+\d+\.?\d*\s+)')


In [94]:
df = df.drop('Aligned length', axis=1)

In [95]:
df = df.apply(pd.to_numeric, errors='ignore')

In [96]:
df.to_csv(snakemake.output[0], index=False)

,File,Length_Chain1,Length_Chain2,TM-score_Chain1,TM-score_Chain2,pdb1,pdb2,Aligned_length,RMSD,Seq_ID
0,../tmp/FATCAT_aligments_twisted/A0A5K4F5N0_A0A...,273.0,281.0,0.71662,0.69835,A0A5K4F5N0,A0A061JES7,232.0,3.24,0.172
1,../tmp/FATCAT_aligments_twisted/A0A1D6PUC8_Q4D...,549.0,213.0,0.30336,0.69365,A0A1D6PUC8,Q4DE48,195.0,4.04,0.087
2,../tmp/FATCAT_aligments_twisted/A0A1D6MYY7_Q4D...,206.0,270.0,0.78385,0.61661,A0A1D6MYY7,Q4DKS2,196.0,3.18,0.117
3,../tmp/FATCAT_aligments_twisted/Q9CPZ8_Tb92793...,168.0,106.0,0.43525,0.64797,Q9CPZ8,Tb92793340,89.0,3.03,0.180
4,../tmp/FATCAT_aligments_twisted/U7PIX3_A0A0M9F...,521.0,444.0,0.43058,0.49219,U7PIX3,A0A0M9FYZ1,299.0,5.67,0.060
...,...,...,...,...,...,...,...,...,...,...
40347,../tmp/FATCAT_aligments_twisted/I1MFF9_Tb92784...,372.0,617.0,0.85773,0.53211,I1MFF9,Tb92784270,356.0,3.03,0.261
40348,../tmp/FATCAT_aligments_twisted/A0A3Q0KH70_Q4D...,235.0,331.0,0.80536,0.59528,A0A3Q0KH70,Q4DHT1,228.0,2.92,0.140
40349,../tmp/FATCAT_aligments_twisted/U7PNP3_A0A504X...,847.0,914.0,0.29389,0.27606,U7PNP3,A0A504XL00,350.0,7.52,0.051
40350,../tmp/FATCAT_aligments_twisted/A0A3P7FJF7_B6D...,302.0,755.0,0.77880,0.32769,A0A3P7FJF7,B6DTI9,263.0,2.67,0.152
